# Trapozoidal Rule

To numerically evaluate an integral, we fit a trapezoid at the kth slice, find its area, and sum up all areas.

Key equation:

$$
I(a,b) = h\left(\frac{1}{2}f(a) + \frac{1}{2}f(b) + \sum_{k=1}^{N-1}f(a+kh)\right)
$$

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def g(x):
    return x**4 - 2*x + 1

def trap_integrate(f,N,a,b): # machine precision at 10e8
    """Integrate a function via trapezoids interpolation"""
    h = (b-a)/N 
    sum_part = 0
    for k in range(1,N):
        sum_part += f(a+k*h)
    left_boundary = f(a)
    right_boundary = f(b)
    integral = h*(0.5*left_boundary + 0.5*right_boundary + sum_part)
    return integral
    
trap_integrate(g, 10, 0.0, 2.0)

4.50656

# Simpson's Rule

We can also use a quadratic fitting for a smoother and more accurate integration.

Key equation:

$$
I(a,b) = \frac{1}{3}h\left(f(a) + f(b) + 4\sum_{1,\text{odd}}^{N-1}f(a+kh) + 2\sum_{2,\text{even}}^Nf(a+kh)\right)
$$


In [4]:
def simpson_integrate(f,N,a,b): # machine precision at 10e4
    """Integrate a function via quadratic curve interpolation"""
    h = (b-a)/N 
    odd_sum_part = 0
    even_sum_part = 0
    for k in range(1,N,2):
        odd_sum_part += f(a+k*h)
    for k in range(2,N-1,2):
        even_sum_part += f(a+k*h)
    left_boundary = f(a)
    right_boundary = f(b)
    integral = (1/3)*h*(left_boundary + right_boundary + 4*odd_sum_part + 2*even_sum_part)
    return integral

simpson_integrate(g,10, 0.0, 2.0)


4.400426666666667

# Adaptive Trapezoidal Rule Method

To properly budget errors, we use the following adaptive integration algorithm.

Key equations:

$$
I(a,b) = h\left(\frac{1}{2}f(a) + \frac{1}{2}f(b) + \sum_{k=1}^{N-1}f(a+kh)\right)
$$

$$
I_i=\frac{1}{2}I_{i-1} + h_i\sum_{k=1,\text{odd}}^{N-1}f(a+kh)
$$

$$
\varepsilon_i = \frac{1}{3}(I_i-I_{i-i})
$$

In [7]:
def adap_trap_integrate(f,initial_steps,a,b,desired_error):
    """Trapezoidal rule budget counter"""
    trap_error = 999 # initializations
    integral = 0
    N = initial_steps
    step_counter = 0
    initial_integration = trap_integrate(f,N,a,b)
    while desired_error < abs(trap_error): # main loop: modify initial integral, then count error
        N = 2*N
        h = (b-a)/N 
        sum_part = 0
        for k in range(1,N,2):
            sum_part += f(a+k*h)
        integral = (1/2)*initial_integration + h*sum_part
        trap_error = (1/3)*(integral - initial_integration)
        initial_integration = integral
        step_counter += 1
    print("Total steps to reach desired error at {} accuracy: {} steps".format(desired_error,step_counter))
    return integral

adap_trap_integrate(g,1,0.0,2.0,0.01) # placeholder for 1 here indicates the start of erroneous digits


Total steps to reach desired error at 0.01 accuracy: 6 steps


4.402604103088379

#  Adaptive Simpson's Rule Method

To properly budget errors, we use the following adaptive integration algorithm.

Key equations:

$$
I(a,b) = \frac{1}{3}h\left(f(a) + f(b) + 4\sum_{1,\text{odd}}^{N-1}f(a+kh) + 2\sum_{2,\text{even}}^Nf(a+kh)\right)
$$

$$
I_i=h_i\left(f(a) + f(b) +2 \sum_{k=2, \text{even}}^{N-2}f(a+kh_i)\right)
+
\frac{4}{3}\sum_{k=1, \text{odd}}^{N-1}f(a+kh_i)
$$

$$
\varepsilon_i = \frac{1}{15}(I_i-I_{i-i})
$$

In [21]:
def adap_simpson_integrate(f,initial_steps,a,b,desired_error):
    """Simpson's rule budget counter"""
    simp_error = 999 # initializations
    integral = 0
    N = initial_steps
    step_counter = 0
    S = 0
    T = 0
    h = (b-a)/N
    even_sum_part = 0
    odd_sum_part = 0
    for k in range(2,N-1,2):
        even_sum_part += f(a+k*h)
    for k in range(1,N,2):
        odd_sum_part += f(a+k*h)
    S = (1/3)*(f(a) + f(b) + 2*even_sum_part)
    T = (2/3)*odd_sum_part
    initial_integration = h*(S + 2*T)
    while desired_error < (simp_error): # main loop: modify initial integral, then count error
        N = 2*N
        h = (b-a)/N
        S = S + T
        odd_sum_part = 0
        for k in range(1,N,2):
            odd_sum_part += f(a+k*h)
        T = (2/3)*odd_sum_part
        integral = h*(S + 2*T)
        simp_error = abs((1/15)*(integral - initial_integration))
        initial_integration = integral
        step_counter += 1
    print("Total steps to reach desired error at {} accuracy: {} steps".format(desired_error,step_counter))
    return integral

    
adap_simpson_integrate(g,1,0.0,2.0,0.01) # placeholder for 1 here indicates the start of erroneous digits

Total steps to reach desired error at 0.01 accuracy: 3 steps


4.401041666666666